# NeuroNexus A1x16 解析パイプライン (v6a)

マウスS1BFバレル皮質からのNeuroNexus A1x16シリコンプローブ記録の解析パイプライン。

## パイプライン概要

| Step | 内容 | モジュール |
|------|------|------------|
| 1 | PLXデータ読み込み | `data_loader.py` |
| 2 | LFP前処理（フィルタ・ICA） | `processing.py` |
| 3 | スパイクソーティング + キュレーション | `spike_sorting.py` |
| 4 | 刺激応答解析（PSTH・適応） | `stimulus.py` |
| 5 | 位相ロック解析 | `spike_lfp_analysis.py` |
| 6 | 保存・可視化 | `saving.py` |
| 7 | 全チャンネル統合解析 | `comprehensive_analysis.py` |

---
## 0. 設定

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

# === パス設定 ===
# 方法1: ファイルダイアログで選択
from get_path import get_path
plx_file = get_path(file_type='plx')

# 方法2: 直接指定
# plx_file = r'path/to/your/recording.plx'

# 出力ディレクトリ（PLXと同じ場所に作成）
output_dir = os.path.join(os.path.dirname(plx_file), 'analysis_output')
os.makedirs(output_dir, exist_ok=True)

print(f'PLX: {plx_file}')
print(f'Output: {output_dir}')

---
## Step 1: データ読み込み

PLXファイルから LFP (1kHz×16ch)、Wideband (40kHz×16ch)、イベントを読み込む。  
チャンネル順序は物理→論理マッピングを自動適用。

In [ ]:
from run_analysis import step1_load_data

# NeuroNexus A1x16-5mm-50-703 のチャンネルマッピング
CHANNEL_ORDER = [8, 7, 9, 6, 12, 3, 11, 4, 14, 1, 15, 0, 13, 2, 10, 5]

session = step1_load_data(plx_file, channel_order=CHANNEL_ORDER)

In [ ]:
# === データ確認 ===
print(f'Recording: {session.basename}')
print(f'Duration:  {session.duration:.1f} s')
print(f'LFP:       {session.lfp_raw.shape} @ {session.fs_lfp} Hz')
print(f'Wideband:  {session.wideband.shape} @ {session.fs_wideband} Hz')
print(f'Stim:      {len(session.stim_times)} events')
print(f'Trials:    {session.n_trials} x {session.n_stim_per_trial} stim @ {session.stim_freq:.0f} Hz')
print(f'ITI:       {session.iti:.1f} s')
print(f'Channels:  {session.channel_order}')

---
## Step 2: LFP前処理

バンドパス (0.1-100Hz) → ノッチ (60Hz) → 高調波除去 (10Hz piezo) → Bad channel検出 → モーション解析 → ICA

In [ ]:
from run_analysis import step2_process_lfp

# デフォルト設定で実行
lfp_result = step2_process_lfp(session)

# カスタム設定が必要な場合:
# from pipeline import PipelineConfig
# config = PipelineConfig(
#     filter_lowcut=0.5,       # 低域カット (Hz)
#     filter_highcut=100,      # 高域カット (Hz)
#     notch_enabled=True,      # 60Hzノッチ
#     harmonic_removal_enabled=True,  # 10Hz高調波除去
#     motion_analysis=True,    # モーション解析
#     ica_enabled=True,        # ICAアーティファクト除去
# )
# lfp_result = step2_process_lfp(session, config=config)

In [ ]:
# === LFP確認プロット ===
lfp = lfp_result['lfp_cleaned']
lfp_times = lfp_result['lfp_times']
fs = lfp_result['fs']

print(f'Cleaned LFP:   {lfp.shape}')
print(f'Good channels: {lfp_result["good_channels"]}')
print(f'Noise ratio:   {100 * np.mean(lfp_result["noise_mask"]):.1f}%')

# 最初の5秒を表示
fig, ax = plt.subplots(figsize=(14, 6))
t_end = min(5.0, lfp_times[-1])
tmask = lfp_times <= t_end
spacing = np.percentile(np.abs(lfp), 98) * 1.5
for i in range(lfp.shape[1]):
    ax.plot(lfp_times[tmask], lfp[tmask, i] - i * spacing, 'k', linewidth=0.4)
# 刺激タイミング
for st in session.stim_times:
    if st <= t_end:
        ax.axvline(st, color='red', alpha=0.3, linewidth=0.5)
ax.set_xlabel('Time (s)')
ax.set_title('Cleaned LFP (first 5s, red = stimulus)')
ax.set_yticks([])
plt.tight_layout()
plt.show()

---
## Step 3: スパイクソーティング

4つのキュレーションモードから選択:

| モード | 説明 |
|--------|------|
| `'auto'` | 基準ベースで自動分類+マージ（GUIなし） |
| `'gui'` | GUIのみ（手動キュレーション） |
| `'both'` | 自動分類 → GUIで確認・微調整 |
| `'none'` | キュレーションなし（生のクラスタリング） |

### 自動キュレーション基準

```
各ユニット
  ├─ SNR < 1.5 or ISI > 15% or n < 30 → Noise ✗
  ├─ ISI > 5%                          → MUA △
  ├─ ISI > 2% かつ SNR < 4             → MUA △
  ├─ ISI < 0.5% かつ SNR > 6           → Excellent SU ◎
  ├─ ISI < 2% かつ SNR > 3             → Good SU ○
  └─ それ以外                           → MUA △

自動マージ: 波形相関 > 0.92 かつ 振幅差 < 25%
```

In [ ]:
from run_analysis import step3_spike_sorting, step3_load_sorting

# ============================================================
#  方法A: 新規ソーティング（全チャンネル、自動キュレーション）
# ============================================================

sorting_results = step3_spike_sorting(
    session,
    channels=None,         # None = 全16チャンネル（[0,1,2,...] で指定も可）
    curation='auto',       # 'auto' / 'gui' / 'both' / 'none'
)

In [ ]:
# ============================================================
#  方法B: キュレーション基準をカスタマイズ
# ============================================================

from spike_sorting import SortingConfig, AutoCurationConfig

# ソーティングパラメータ
sort_config = SortingConfig(
    threshold_std=4.0,      # スパイク検出閾値（sigma倍）
    max_clusters=5,         # 最大クラスター数
    min_cluster_size=20,    # 最小クラスターサイズ
)

# 自動キュレーション基準
auto_config = AutoCurationConfig(
    # --- Noise判定 ---
    noise_snr_max=1.5,           # SNR < 1.5 → Noise
    noise_isi_min=15.0,          # ISI違反 > 15% → Noise
    noise_min_spikes=30,         # スパイク数 < 30 → Noise
    
    # --- MUA判定 ---
    mua_isi_min=5.0,             # ISI > 5% → 必ずMUA
    mua_isi_soft=2.0,            # ISI > 2% かつ SNR < mua_snr_max → MUA
    mua_snr_max=4.0,
    
    # --- SU判定 ---
    su_good_isi_max=2.0,         # ISI < 2% かつ SNR > 3 → Good SU
    su_good_snr_min=3.0,
    su_excellent_isi_max=0.5,    # ISI < 0.5% かつ SNR > 6 → Excellent
    su_excellent_snr_min=6.0,
    
    # --- 自動マージ ---
    merge_enabled=True,          # 類似波形ユニットの自動統合
    merge_waveform_corr_min=0.92,  # 波形相関閾値
    merge_amplitude_ratio_max=0.25,  # 振幅差閾値
    
    # --- 発火率フィルタ ---
    min_firing_rate_hz=0.0,      # 0 = 無効
)

sorting_results = step3_spike_sorting(
    session,
    config=sort_config,
    auto_config=auto_config,
    curation='auto',
)

In [ ]:
# ============================================================
#  方法C: 自動キュレーション → GUIで確認・微調整
# ============================================================

# sorting_results = step3_spike_sorting(
#     session,
#     curation='both',  # 自動分類してからGUIが開く
# )

In [ ]:
# ============================================================
#  方法D: GUIのみ（従来どおり手動で全部やる）
# ============================================================

# sorting_results = step3_spike_sorting(
#     session,
#     channels=[0],      # まず1チャンネルだけ試す場合
#     curation='gui',
# )

In [ ]:
# ============================================================
#  方法E: 保存済みソーティングを読み込み
# ============================================================

# sorting_results = step3_load_sorting('sorting_ch0.npz')

# 読み込み後に自動キュレーションだけ適用することも可能:
# from spike_sorting import auto_curate_all, AutoCurationConfig
# auto_curate_all(sorting_results, AutoCurationConfig(),
#                 recording_duration=session.duration)

In [ ]:
# === ソーティング結果の保存 ===
from spike_sorting import save_sorting_results

sorting_path = os.path.join(output_dir, f'{session.basename}_sorting.npz')
save_sorting_results(sorting_results, sorting_path)

In [ ]:
# === ソーティング結果の確認テーブル ===
from spike_sorting import classify_unit, AutoCurationConfig

ac = AutoCurationConfig()
marks = {'noise': 'x', 'mua': 'triangle', 'good_su': 'o', 'excellent_su': 'star'}

print(f'{"Ch":>4} {"Unit":>5} {"nSpk":>6} {"SNR":>6} {"ISI%":>6} {"FR":>7} {"Quality":>14}')
print('-' * 55)
for ch in sorted(sorting_results.keys()):
    for unit in sorting_results[ch].units:
        quality = classify_unit(unit, ac, session.duration)
        mark = {'noise': 'x', 'mua': 'triangle', 'good_su': 'o', 'excellent_su': '*'}[quality]
        dur = (unit.spike_times[-1] - unit.spike_times[0]) if len(unit.spike_times) > 1 else 1
        fr = unit.n_spikes / dur if dur > 0 else 0
        print(f'{ch:>4} {unit.unit_id:>5} {unit.n_spikes:>6} '
              f'{unit.snr:>6.2f} {unit.isi_violation_rate:>6.2f} {fr:>7.2f} [{mark}] {quality}')

---
## Step 4: 刺激応答解析

PSTH、ラスタープロット、刺激適応（10発の応答減弱）を計算。

In [ ]:
from run_analysis import step4_stimulus_analysis

protocol, stim_results = step4_stimulus_analysis(
    session, sorting_results, lfp_result
)

In [ ]:
# === 個別ユニットの刺激応答プロット ===

for unit_key, res in stim_results.items():
    print(f'\n{unit_key}: peak={res["psth"].peak_latency_ms:.1f}ms, '
          f'adaptation={res["adaptation"].adaptation_ratio:.2f}')
    
    fig, axes = plt.subplots(1, 3, figsize=(14, 3))
    protocol.plot_psth(res['spike_times'], ax=axes[0])
    axes[0].set_title(f'{unit_key} PSTH')
    protocol.plot_raster(res['spike_times'], ax=axes[1])
    axes[1].set_title('Raster')
    protocol.plot_adaptation(res['spike_times'], ax=axes[2])
    axes[2].set_title('Adaptation')
    plt.tight_layout()
    plt.show()

---
## Step 5: 位相ロック解析

各ユニットのスパイクがLFPのどの位相にロックしているかを解析。  
帯域: delta (1-4Hz), theta (4-8Hz), alpha (8-14Hz), beta (14-30Hz), gamma (30-100Hz)

In [ ]:
from run_analysis import step5_phase_locking

# デフォルト帯域
analyzer = step5_phase_locking(
    sorting_results, lfp_result, protocol
)

# カスタム帯域:
# analyzer = step5_phase_locking(
#     sorting_results, lfp_result, protocol,
#     freq_bands={
#         'theta': (4, 8),
#         'beta':  (14, 30),
#         'gamma': (30, 80),
#     },
#     min_spikes=50,
# )

In [ ]:
# === 位相ロック結果テーブル ===

print(f'{"Unit":>15} {"Band":>10} {"MRL":>6} {"PPC":>6} {"p":>8} {"Sig":>4}')
print('-' * 55)
for unit_key, pl in analyzer.unit_results.items():
    for band, ch_results in pl.band_results.items():
        best = max(
            (r for r in ch_results.values() if r is not None),
            key=lambda r: r.mrl, default=None
        )
        if best:
            sig = '***' if best.p_value < 0.001 else (
                  '**' if best.p_value < 0.01 else (
                  '*' if best.significant else ''))
            print(f'{unit_key:>15} {band:>10} {best.mrl:>6.3f} '
                  f'{best.ppc:>6.3f} {best.p_value:>8.4f} {sig:>4}')

In [ ]:
# === ユニットサマリープロット（9パネル） ===

for unit_key, pl in analyzer.unit_results.items():
    fig = analyzer.plot_unit_summary(pl.channel, pl.unit_id)
    plt.show()

In [ ]:
# === 集団サマリー（6パネル） ===
fig = analyzer.plot_population_summary()
plt.show()

---
## Step 6: 保存

ソーティング結果（NPZ）、位相ロック結果（CSV）、プロット（PNG）を一括保存。

In [ ]:
from run_analysis import step6_save_and_plot

step6_save_and_plot(
    analyzer, sorting_results, protocol,
    output_dir, session.basename
)

---
## Step 7: 全チャンネル統合解析

16チャンネルを横断的に解析し、深度プロファイル・CSD・コヒーレンスなどを算出。

### 出力一覧

| プロット | 内容 |
|----------|------|
| `spike_overview.png` | 全チャンネル波形 + 品質散布図 + 発火率深度 |
| `firing_rate_condition.png` | Baseline/Stim/Post 発火率×深度 |
| `csd_summary.png` | Trial平均CSD + LFP + 深度断面 |
| `lfp_depth_analysis.png` | パワー深度 + コヒーレンス |
| `phase_locking_depth.png` | MRL帯域×ユニット + MRL深度 |
| `grand_summary.png` | 12パネル統合サマリー |
| `unit_quality_table.csv` | 品質テーブル |
| `analysis_report.txt` | テキストレポート |

In [ ]:
from comprehensive_analysis import ComprehensiveAnalyzer

ca = ComprehensiveAnalyzer(
    session=session,
    lfp_result=lfp_result,
    sorting_results=sorting_results,
    protocol=protocol,
    spike_lfp_analyzer=analyzer,
)

### 7-1. スパイクソーティング概要

In [ ]:
# 品質テーブル
table = ca.spike_sorting_table()

try:
    import pandas as pd
    df = pd.DataFrame(table)
    display(df)
except ImportError:
    print(f'{"Ch":>4} {"Depth":>6} {"Layer":>5} {"Unit":>5} '
          f'{"nSpk":>6} {"SNR":>6} {"ISI%":>6} {"FR":>7} {"Quality":>12}')
    print('-' * 70)
    for r in table:
        print(f'{r["channel"]:>4} {r["depth_um"]:>6} {r["layer"]:>5} '
              f'{r["unit_id"]:>5} {r["n_spikes"]:>6} {r["snr"]:>6.2f} '
              f'{r["isi_violation"]:>6.2f} {r["firing_rate_hz"]:>7.2f} '
              f'{r["quality"]:>12}')

In [ ]:
# 全チャンネル波形 + 品質散布図 + 発火率深度
fig = ca.plot_spike_overview()
plt.show()

In [ ]:
# 条件別発火率×深度
fig = ca.plot_firing_rate_by_condition()
plt.show()

### 7-2. LFP深度解析

In [ ]:
# LFP波形概要（時間範囲は指定可）
fig = ca.plot_lfp_overview(t_range=(0, 5))
plt.show()

In [ ]:
# CSD (Current Source Density)
# 赤=source（電流湧き出し）、青=sink（電流吸い込み）
fig = ca.plot_csd_summary(window_ms=(-50, 200))
plt.show()

In [ ]:
# パワースペクトル深度ヒートマップ + 帯域別パワー + コヒーレンス
fig = ca.plot_lfp_depth_analysis()
plt.show()

### 7-3. スパイク-LFP統合解析

In [ ]:
# 位相ロック×深度プロファイル
fig = ca.plot_phase_locking_depth()
plt.show()

In [ ]:
# STA深度プロファイル
fig = ca.plot_sta_depth()
plt.show()

### 7-4. グランドサマリー（12パネル統合図）

In [ ]:
fig = ca.plot_grand_summary()
plt.show()

### 7-5. 一括保存

In [ ]:
comp_dir = os.path.join(output_dir, 'comprehensive')
ca.save_all(comp_dir)

---
## 一括実行（Step 1-7 をまとめて）

上のセルを個別に実行する代わりに、全ステップを一気に回すことも可能。

In [ ]:
# from run_analysis import run_full_pipeline
#
# results = run_full_pipeline(
#     plx_file=plx_file,
#     output_dir=output_dir,
#     channel_order=CHANNEL_ORDER,
#     curation='auto',           # 'auto' / 'gui' / 'both' / 'none'
#     # sorting_file='sorting.npz',  # 既存のソーティングを使う場合
# )
#
# # 個別の結果にアクセス
# session          = results['session']
# lfp_result       = results['lfp_result']
# sorting_results  = results['sorting_results']
# protocol         = results['protocol']
# analyzer         = results['analyzer']
# ca               = results['comprehensive']

---
## 便利ツール

### CSDの生データにアクセス

In [ ]:
# 連続CSD
csd, csd_depths = ca.compute_csd(sigma=1.0)
print(f'CSD: {csd.shape}, depths: {csd_depths} um')

# Trial平均CSD（時間窓カスタム）
avg_csd, csd_times_ms, _ = ca.compute_trial_averaged_csd(window_ms=(-100, 300))
print(f'Trial-avg CSD: {avg_csd.shape}')

### パワースペクトル

In [ ]:
freqs, psd = ca.compute_power_by_depth()

fig, ax = plt.subplots(figsize=(8, 5))
freq_mask = freqs <= 100
for i in range(psd.shape[1]):
    ax.semilogy(freqs[freq_mask], psd[freq_mask, i],
                alpha=0.5, label=f'{ca.depths[i]}um')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('PSD')
ax.set_title('Power Spectral Density by Channel')
ax.legend(fontsize=6, ncol=4)
plt.tight_layout()
plt.show()

### コヒーレンス行列（任意の帯域）

In [ ]:
coh_theta = ca._compute_coherence_matrix((4, 8))
coh_gamma = ca._compute_coherence_matrix((30, 80))

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
for ax, coh, title in [(axes[0], coh_theta, 'Theta (4-8 Hz)'),
                        (axes[1], coh_gamma, 'Gamma (30-80 Hz)')]:
    im = ax.imshow(coh, cmap='viridis', vmin=0, vmax=1)
    ax.set_title(f'Coherence: {title}')
    plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()

### 自動キュレーションを後から再適用

In [ ]:
from spike_sorting import auto_curate_all, AutoCurationConfig

# 例: 厳しい基準に変更
strict = AutoCurationConfig(
    su_good_snr_min=5.0,
    su_good_isi_max=1.0,
    merge_enabled=False,
)

# 注意: sorting_results のユニットが直接変更される
summary = auto_curate_all(
    sorting_results, strict,
    recording_duration=session.duration
)
print(f"SU: {summary['total_su']}, MUA: {summary['total_mua']}, "
      f"Noise: {summary['total_noise']}, Excellent: {summary['total_excellent']}")

### GUIツール

In [ ]:
# === 波形ブラウザ ===
# from waveform_browser import WaveformBrowser
# wb = WaveformBrowser(sorting_results)
# wb.run()

# === ソーティングGUI（後から修正） ===
# from spike_sorting_gui import SpikeSortingGUI
# gui = SpikeSortingGUI(sorting_results)
# gui.run()
# sorting_results = gui.results

# === LFPパイプライン設定GUI ===
# from config_gui import run_config_gui
# config = run_config_gui()

---
## 出力ファイル構造

```
analysis_output/
|
|-- {basename}_sorting.npz           # ソーティング結果
|-- {basename}_spike_times.csv       # スパイク時刻
|-- {basename}_phase_locking.csv     # 位相ロック（帯域 x チャンネル）
|-- {basename}_conditions.csv        # 条件別位相ロック
|-- {basename}_unit_summary.csv      # ユニットサマリー
|
|-- plots/
|   |-- ch*_unit*_summary.png        # ユニット別9パネル
|   |-- ch*_unit*_stimulus.png       # 刺激応答
|   +-- *_population_summary.png     # 集団サマリー
|
+-- comprehensive/
    |-- spike_overview.png           # 全チャンネル波形+品質+発火率
    |-- firing_rate_condition.png    # 条件別発火率深度
    |-- csd_summary.png             # CSD
    |-- lfp_depth_analysis.png      # パワー+コヒーレンス
    |-- lfp_overview.png            # LFP波形
    |-- phase_locking_depth.png     # 位相ロック深度
    |-- sta_depth.png               # STA深度
    |-- grand_summary.png           # 12パネル統合
    |-- unit_quality_table.csv      # 品質テーブル
    +-- analysis_report.txt         # テキストレポート
```